In [12]:
import ROOT
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [13]:
def get_files_in_range(directory, min_run, max_run):
    return_list = []
    for filename in os.listdir(directory):
        if filename.startswith('sidisdvcs'):
            run = filename.split('_')[1].split('.')[0]
            if min_run <= int(run) <= max_run:
                return_list.append(os.path.join(directory, filename))
    return sorted(return_list, key=lambda x: int(x.split('_')[1].split('.')[0]))

In [14]:
def setAxes(tg,axes):
    tg.GetXaxis().SetLimits(axes[0],axes[1])
    tg.GetYaxis().SetRangeUser(axes[2],axes[3])
    tg.GetYaxis().SetLimits(axes[2],axes[3])
    return tg

In [15]:
# Assign unique markers and colors to array of TGraphErrors
def stylize_tgrapherrors(tge):
    nMarkers = len(TAttMarkers)
    nColors = len(TColors)
    nPlots = len(tge)
    if(nPlots > nColors * nMarkers):
        print("ERROR: The number of plots sent to stylize_tgrapherrors is too large (",nPlots,">",nColors*nMarkers,")...Aborting...")
        return -1
    
    for i in range(nColors):
        color = TColors[i]
        for j in range(nMarkers):
            marker = TAttMarkers[j]
            if(nMarkers*i+j>=nPlots):
                return tge
            else:
                tge[nMarkers*i+j].SetMarkerStyle(marker)
                tge[nMarkers*i+j].SetMarkerColor(color)
                tge[nMarkers*i+j].SetMarkerColor(color)
                tge[nMarkers*i+j].SetMarkerSize(MarkerSize)  
    return tge

In [16]:
# From an array of plots, get the minimum and maximum y values
# The 'error' parameter is for error bar plots
def get_plot_minmax(plotArr,error, expand = 0.1):
    plotmin, plotmax = 999,-999
    for plot in plotArr:
        for point_idx in range(plot.GetN()):
            tempUp, tempDown = 0,0
            if(error==True):
                tempUp = plot.GetPointY(point_idx) + plot.GetErrorY(point_idx)
                tempDown = plot.GetPointY(point_idx) - plot.GetErrorY(point_idx)
            else:
                tempUp = plot.GetPointY(point_idx)
                tempDown = tempUp
            
            if(tempUp>plotmax):
                plotmax = tempUp
            if(tempDown<plotmin):
                plotmin = tempDown
                
    # Expand range by expand%
    plotRange = plotmax-plotmin
    exp = plotRange * expand
    plotmin = plotmin - 0.5 * exp
    plotmax = plotmax + 0.5 * exp
    
    # Return
    return plotmin,plotmax            

In [17]:
def calc_Pt(nruns,reps,Np,Np_err,Nm,Nm_err,A_th,beamPol,f,ferr):
    Pt_Arr = []
    Pt_err_Arr = []
    
    for i in range(nruns):
        
        _s = slice(i*reps,(i+1)*reps)
        
        numerator = ufloat(0,0)
        denominator = ufloat(0,0)
        for np,nm,np_err,nm_err,ath,df,df_err in zip(Np[_s],Nm[_s],Np_err[_s],Nm_err[_s],A_th[_s],f[_s],ferr[_s]):
            
            np = ufloat(np,np_err)
            nm = ufloat(nm,nm_err)
            df = ufloat(df,df_err)
            
            numerator+=(np-nm)*ath*df
            denominator+=(np+nm)*ath*ath*df*df
            
        Pt = numerator/denominator/beamPol
        
        Pt_Arr.append(Pt.n)
        Pt_err_Arr.append(Pt.s)

    return Pt_Arr, Pt_err_Arr

In [18]:
def get_single_run_hists(file,targ):
    
    # Get run number
    run = file.split('_')[1].split('.')[0]
    
    
    # Get TFile and important objects
    tfile = ROOT.TFile(file,"READ")
    hwp = tfile.Get("hwp")[0]
    rcdb_tpol = tfile.Get("rcdb_tpol")[0]
    fcup_pos = tfile.Get("fcup_pos")[0]
    fcup_neg = tfile.Get("fcup_neg")[0]
    beamE = tfile.Get("beamE")[0]
    target = tfile.Get("target")
    if(targ!=target):
        print(targ,target)
        return -1
    

    # Deal with HWP and Tpol sign (HWP out = 1)
    zhat = 0
    if(hwp==1 and rcdb_tpol>0):
        zhat = 1
    elif(hwp==1 and rcdb_tpol<0):
        zhat = -1
        fcup_pos,fcup_neg=fcup_neg,fcup_pos
    elif(hwp==0 and rcdb_tpol>0):
        zhat = -1
        fcup_pos,fcup_neg=fcup_neg,fcup_pos
    elif(hwp==0 and rcdb_tpol<0):
        zhat = 1
    else:
        print("FCup error?")
        return -1
    
    fcup_parallel, fcup_antiparallel = fcup_pos,fcup_neg
    
    # Create the RDataFrame
    df = ROOT.RDataFrame("events",file)
    
    # Create the basic DIS cuts
    df = df.Define("vz","abs(Vz+4.5)").Filter("E > 2.6 && Th > 0.14 && Th < 0.611 && vz < 4")
    
    # Pull Sebastian's binning
    A_LL_proton_sebastian = pd.read_csv("./sebastian_A_LL_p.txt",sep=" ",names=["xmin","xmax","Q2min","Q2max","A_LL"])
    A_LL_deuteron_sebastian = pd.read_csv("./sebastian_A_LL_d.txt",sep=" ",names=["xmin","xmax","Q2min","Q2max","A_LL"])
    
    # Pull x,Q2 bins from Sebastian's tables
    xbins,Q2bins=0,0
    if(target=="NH3"):
        xbins = np.unique(np.array(A_LL_proton_sebastian["xmin"].to_list()+A_LL_proton_sebastian["xmax"].to_list()))
        Q2bins = np.unique(np.array(A_LL_proton_sebastian["Q2min"].to_list()+A_LL_proton_sebastian["Q2max"].to_list()))
    elif(target=="ND3"):
        xbins = np.unique(np.array(A_LL_deuteron_sebastian["xmin"].to_list()+A_LL_deuteron_sebastian["xmax"].to_list()))
        Q2bins = np.unique(np.array(A_LL_deuteron_sebastian["Q2min"].to_list()+A_LL_deuteron_sebastian["Q2max"].to_list()))
    else:
        xbins = np.array([0.0,1.0])
        Q2bins = np.array([0.0,100.0])
            
    # Create 2d histograms for x,Q2 binning
    # Do this for when the target+beam are aligned, and another where they are anti-aligned
    h_p = df.Filter("hel=={}".format(zhat)).Histo2D(("h_p","",len(xbins)-1,xbins,len(Q2bins)-1,Q2bins),"x","Q2")
    h_ap = df.Filter("hel=={}".format(-zhat)).Histo2D(("h_p","",len(xbins)-1,xbins,len(Q2bins)-1,Q2bins),"x","Q2")
    
    # Process histograms (from Histo1D --> TH1D)
    H_ap = h_ap.GetValue().Clone()
    H_p = h_p.GetValue().Clone()
    Hw_ap = h_ap.GetValue().Clone() # To be weighted by beam Charge Asym
    Hw_p = h_p.GetValue().Clone()   # To be weighted by beam Charge Asym
    
    H_ap.Sumw2()
    H_p.Sumw2()
    Hw_ap.Sumw2()
    Hw_p.Sumw2()
    
    
    Hw_ap.Scale(1/fcup_antiparallel)
    Hw_p.Scale(1/fcup_parallel)
    
    # Grab N+ and N- along with their errors
    N_p = [H_p.GetBinContent(i+1,j+1) for i in range(H_p.GetNbinsX()) for j in range(H_p.GetNbinsY())]
    N_m = [H_ap.GetBinContent(i+1,j+1) for i in range(H_p.GetNbinsX()) for j in range(H_p.GetNbinsY())]
    Nw_p = [Hw_p.GetBinContent(i+1,j+1) for i in range(H_p.GetNbinsX()) for j in range(H_p.GetNbinsY())]
    Nw_m = [Hw_ap.GetBinContent(i+1,j+1) for i in range(H_p.GetNbinsX()) for j in range(H_p.GetNbinsY())]
    
    N_err_p = [H_p.GetBinError(i+1,j+1) for i in range(H_p.GetNbinsX()) for j in range(H_p.GetNbinsY())]
    N_err_m = [H_ap.GetBinError(i+1,j+1) for i in range(H_p.GetNbinsX()) for j in range(H_p.GetNbinsY())]
    Nw_err_p = [Hw_p.GetBinError(i+1,j+1) for i in range(H_p.GetNbinsX()) for j in range(H_p.GetNbinsY())]
    Nw_err_m = [Hw_ap.GetBinError(i+1,j+1) for i in range(H_p.GetNbinsX()) for j in range(H_p.GetNbinsY())]
    
    # Scale by float to avoid division issues
    H_ap.Scale(1.0)
    H_p.Scale(1.0)
    
    # Create numerator and denominator histogram
    H_numerator = H_p.Clone()
    H_denominator = H_p.Clone()
    Hw_numerator = Hw_p.Clone()
    Hw_denominator = Hw_p.Clone()
    
    # Construct numerator and denominator of asymmetry
    H_numerator.Add(H_ap,-1)
    H_denominator.Add(H_ap,1)
    Hw_numerator.Add(Hw_ap,-1)
    Hw_denominator.Add(Hw_ap,1)
    
    # Divide numerator and denominator
    H = H_numerator.Clone()
    H.Divide(H_denominator)
    Hw = Hw_numerator.Clone()
    Hw.Divide(Hw_denominator)
    
    # Label Histogram
    H.SetTitle("{} asymmetries RG-C;{};{}".format(target,"x","Q^{2}[GeV^{2}]")+"(N^{+}-N^{-})/(N^{+}+N^{-})")
    Hw.SetTitle("{} asymmetries RG-C;{};".format(target,"x","Q^{2}[GeV^{2}]")+"(L^{-}N^{+}-L^{+}N^{-})/(L^{-}N^{+}+L^{+}N^{-})")
    
    # Return 
    return [run,target,rcdb_tpol,hwp,H.Clone(),Hw.Clone(),fcup_parallel,fcup_antiparallel,N_p,N_m,N_err_p,N_err_m,Nw_p,Nw_m,Nw_err_p,Nw_err_m]

In [19]:
def unpack_to_dataframe(H):
    
    runNumberArr = [a[0] for a in H]
    TargArr = [a[1] for a in H]
    TpolArr = [a[2] for a in H]
    HWPArr = [a[3] for a in H]
    histArr = [a[4] for a in H]
    histwArr = [a[5] for a in H]
    fcup_parallelArr = [a[6] for a in H]
    fcup_antiparallelArr = [a[7] for a in H]
    NpArr = [a[8] for a in H]
    NmArr = [a[9] for a in H]
    NperrArr = [a[10] for a in H]
    NmerrArr = [a[11] for a in H]
    NwpArr = [a[12] for a in H]
    NwmArr = [a[13] for a in H]
    NwperrArr = [a[14] for a in H]
    NwmerrArr = [a[15] for a in H]
    xbins = np.unique(np.array(A_LL_proton_sebastian["xmin"].to_list()+A_LL_proton_sebastian["xmax"].to_list()))
    ybins = np.unique(np.array(A_LL_proton_sebastian["Q2min"].to_list()+A_LL_proton_sebastian["Q2max"].to_list()))
    
    # Create column names for pandas dataframe
    cols = ["Run","Target","Tpol","HWP","xmin","xmax","x","Q2min","Q2max","Q2","A_LL","A_LL_err","A_LL_wt","A_LL_wt_err","A_||","f","ferr","fcup_parallel","fcup_antiparallel","N+","N-","N+err","N-err","n+","n-","n+err","n-err"]
    
    # Create Pandas Dataframe
    df = pd.DataFrame(columns=cols)
    
    # Append rows to dataframe
    for r,targ,tpol,hwp,h,hw,Np,Nn,_np,_nm,_nwp,_nwm,_nperr,_nmerr,_nwperr,_nwmerr in zip(runNumberArr, TargArr, TpolArr, HWPArr,histArr,histwArr,
                                                                                                        fcup_parallelArr,fcup_antiparallelArr,
                                                                                                        NpArr,NmArr,NwpArr,NwmArr,
                                                                                                        NperrArr,NmerrArr,NwperrArr,NwmerrArr):
        row = []
        bin_idx = -1
        for xbin_idx in range(h.GetNbinsX()):
            for ybin_idx in range(h.GetNbinsY()):
                bin_idx+=1
                
                xl = xbins[xbin_idx]
                xr = xbins[xbin_idx+1]
                xc = (xl+xr)/2
                
                yl = ybins[ybin_idx]
                yr = ybins[ybin_idx+1]
                yc = (yl+yr)/2

                a_ll = h.GetBinContent(xbin_idx+1,ybin_idx+1)
                a_ll_err = h.GetBinError(xbin_idx+1,ybin_idx+1)

                a_ll_wt = hw.GetBinContent(xbin_idx+1,ybin_idx+1)
                a_ll_wt_err = hw.GetBinError(xbin_idx+1,ybin_idx+1)
    
                a_ll_theory = []
                if(targ=="NH3"):
                    a_ll_theory = A_LL_proton_sebastian[(A_LL_proton_sebastian["xmin"]==xl) &
                                                        (A_LL_proton_sebastian["Q2min"]==yl)]["A_LL"].to_list()
                elif(targ=="ND3"):
                    a_ll_theory = A_LL_deuteron_sebastian[(A_LL_deuteron_sebastian["xmin"]==xl) &
                                                        (A_LL_deuteron_sebastian["Q2min"]==yl)]["A_LL"].to_list()
                else:
                    a_ll_theory = [0]
                if(a_ll_theory==[]):
                    continue # Do not care about x-Q2 bins with no theory A_LL
                else:
                    a_ll_theory = a_ll_theory[0]
                
                f,ferr=0,0
                dilutiondf=pd.read_csv("/work/clas12/users/gmat/rgc_dilution_factor_v4.csv")
                if(TargArr[0]=="NH3"):
                    f = dilutiondf[(dilutiondf.xmin==xl)&(dilutiondf.Q2min==yl)].df_NH3.values
                    ferr = dilutiondf[(dilutiondf.xmin==xl)&(dilutiondf.Q2min==yl)].df_err_NH3.values
                    if(f.size==0):
                        continue
                elif(TargArr[0]=="ND3"):
                    f = dilutiondf[(dilutiondf.xmin==xl)&(dilutiondf.Q2min==yl)].df_ND3.values
                    ferr = dilutiondf[(dilutiondf.xmin==xl)&(dilutiondf.Q2min==yl)].df_err_ND3.values
                    if(f.size==0):
                        continue
                if(hwp==0):
                    hwp="in"
                else:
                    hwp="out"
                
 
                row = [r,targ,tpol,hwp,xl,xr,xc,yl,yr,yc,a_ll,a_ll_err,a_ll_wt,a_ll_wt_err,a_ll_theory,
                       f,ferr,Np,Nn,
                       _np[bin_idx],_nm[bin_idx],_nperr[bin_idx],_nmerr[bin_idx],
                       _nwp[bin_idx],_nwm[bin_idx],_nwperr[bin_idx],_nwmerr[bin_idx]]
                
                # Add row
                df.loc[len(df.index)] = row

    # Return dataframe
    return df

In [20]:
font = {'size'   : 15}
plt.rc('font', **font)

markerColors=['k','r','b','g','y']
markerStyles=['.','o','v','^','<','>','s','P','*','X','x','d']
markers = [mc+ms for mc in markerColors for ms in markerStyles]

plt.rcParams['text.usetex'] = True
def multigraph(df,beamPol):
    
    # Extract arrays
    runNumberArr = df["Run"].to_numpy()
    TargArr = df["Target"].to_list()
    #if(TargArr[0]=="C"):
    #    multigraph_C(df,beamPol)
    TpolArr = df["Tpol"].to_numpy()
    HWPArr = df["HWP"].to_list()
    xminArr = df["xmin"].to_numpy()
    xmaxArr = df["xmax"].to_numpy()
    xArr = df["x"].to_list()
    Q2minArr = df["Q2min"].to_numpy()
    Q2maxArr = df["Q2max"].to_numpy()
    Q2Arr = df["Q2"].to_list()
    A_Arr = df["A_LL"].to_numpy()
    A_err_Arr = df["A_LL_err"].to_numpy()
    A_wt_Arr = df["A_LL_wt"].to_numpy()
    A_wt_err_Arr = df["A_LL_wt_err"].to_numpy()
    A_th_Arr = df["A_||"].to_numpy()
    dilution_Arr = df["f"].to_numpy()
    dilution_err_Arr = df["ferr"].to_numpy()
    FCup_parallelArr = df["fcup_parallel"].to_numpy() 
    FCup_antiparallelArr = df["fcup_antiparallel"].to_numpy() 
    Np_Arr = df["N+"].to_numpy()
    Nm_Arr = df["N-"].to_numpy()
    Nwp_Arr = df["n+"].to_numpy()
    Nwm_Arr = df["n-"].to_numpy()
    Np_err_Arr = df["N+err"].to_numpy()
    Nm_err_Arr = df["N-err"].to_numpy()
    Nwp_err_Arr = df["n+err"].to_numpy()
    Nwm_err_Arr = df["n-err"].to_numpy()
    
    
    # Extract unique arrays (b/c of repetitive vals)
    reps = np.sum(TpolArr==TpolArr[0]) # Number of kinematic bins (e.g. xbins)
    urunNumberArr = runNumberArr[::reps]
    uTargArr = TargArr[::reps]
    uTpolArr = TpolArr[::reps]
    uHWPArr = HWPArr[::reps]
    uFCup_parallelArr = FCup_parallelArr[::reps]
    uFCup_antiparallelArr = FCup_antiparallelArr[::reps]
    nruns = len(uTargArr)
    print(urunNumberArr)
    
    # Bools for selection
    bool_negPol = (uTpolArr<0)
    bool_HWPin  = np.array([(hwp=="in") for hwp in uHWPArr])
    bool_HWPout = np.array([(hwp=="out") for hwp in uHWPArr])
    
    
    fig1, axs1 = plt.subplots(1,2,figsize=(20, 10))
    axs1[0].ticklabel_format(useOffset=False)
    axs1[1].ticklabel_format(useOffset=False)
    # Tpol vs. run
    axs1[0].plot(urunNumberArr[bool_HWPin],uTpolArr[bool_HWPin],"k.",label="HWP in")
    axs1[0].plot(urunNumberArr[(bool_negPol * bool_HWPin)],-uTpolArr[(bool_negPol * bool_HWPin)],"r.")
    axs1[0].plot(urunNumberArr[bool_HWPout],uTpolArr[bool_HWPout],"kx",label="HWP out")
    axs1[0].plot(urunNumberArr[(bool_negPol * bool_HWPout)],-uTpolArr[(bool_negPol * bool_HWPout)],"rx")
    axs1[0].set_xlabel("Run")
    axs1[0].set_ylabel("Tpol")
    axs1[0].grid()
    axs1[0].legend()

    # Faraday cup ratio vs. run
    uFCup_ratio = (uFCup_parallelArr-uFCup_antiparallelArr)/(uFCup_parallelArr+uFCup_antiparallelArr)
    axs1[1].plot(urunNumberArr[bool_HWPin],uFCup_ratio[bool_HWPin],"k.",label="HWP in")
    axs1[1].plot(urunNumberArr[bool_HWPout],uFCup_ratio[bool_HWPout],"kx",label="HWP out")
    axs1[1].set_xlabel("Run")
    axs1[1].set_ylabel("(FC+  -  FC-)/(FC+  +  FC-)")
    axs1[1].grid()
    axs1[1].legend()

    # Dilution
    fig6,axs6 = plt.subplots(1,1,figsize=(20,10))
    Nruns = len(urunNumberArr)
    for i,r,xmin,xmax,f,ferr in zip(range(len(runNumberArr)),runNumberArr,xminArr,xmaxArr,dilution_Arr,dilution_err_Arr):
        idx = np.floor(i/reps)
        xrange = xmax-xmin
        xcenter = (xmax+xmin)/2
        x = xcenter
        y = f
        yerr = ferr
        
        if(i%nruns==0):
            axs6.errorbar(x,y,yerr=yerr,fmt="ko-")
        else:
            continue
    axs6.grid()
    axs6.set_xlabel("x",fontsize=20)
    axs6.set_ylabel(r"Dilution factor",fontsize=30)
    
    # Target Polarization vs. Run Number
    Nruns = len(urunNumberArr)
    fig8,axs8 = plt.subplots(1,1,figsize=(20,10))
    axs8.ticklabel_format(useOffset=False)
    Pt_w_Arr, Pt_w_err_Arr = calc_Pt(nruns,reps,Nwp_Arr,Nwp_err_Arr,Nwm_Arr,Nwm_err_Arr,A_th_Arr, beamPol,dilution_Arr,dilution_err_Arr)
    Pt_Arr, Pt_err_Arr = calc_Pt(nruns,reps,Np_Arr,Np_err_Arr,Nm_Arr,Nm_err_Arr,A_th_Arr, beamPol,dilution_Arr,dilution_err_Arr)
    
    axs8.errorbar(urunNumberArr,Pt_w_Arr,yerr=Pt_w_err_Arr,label="FCup weights",fmt="ro-")
    axs8.errorbar(urunNumberArr,Pt_Arr,yerr=Pt_err_Arr,label="No FCup weights",fmt="ko-")
    axs8.grid()
    axs8.legend()
    axs8.set_xlabel("Run",fontsize=20)
    axs8.set_ylabel(r"$P_{t}=\frac{1}{P_b}\frac{\sum_{x,Q^{2}}\left[(N^{+}-N^{-}) A_{||}(x,Q^{2})f(x)\right]}{\sum_{x,Q^2}\left[(N^{+}+N^{-}) A^{2}_{||}(x,Q^{2})f^{2}(x)\right]}$",fontsize=30)
    axs8.text(0.15,0.25,r"$P_b={:.1f}\%$".format(100*beamPol),fontsize=25,horizontalalignment='center',
     verticalalignment='center',transform=axs8.transAxes)
    axs8.set_ylim(0,0.5)
    
    fig9,axs9 = plt.subplots(1,1,figsize=(12,10))
    rmin = np.amin(urunNumberArr)
    rmax = np.amax(urunNumberArr)
    axs9.ticklabel_format(useOffset=False)
    boolTpolPos = np.array([_t > 0 for _t in _rcdb_Tpol]) * np.array([_r >= rmin and _r <= rmax for _r in _rcdb_run])
    boolTpolNeg = np.array([_t < 0 for _t in _rcdb_Tpol]) * np.array([_r >= rmin and _r <= rmax for _r in _rcdb_run])
    axs9.errorbar(urunNumberArr,Pt_w_Arr,yerr=Pt_w_err_Arr,label="FCup weights",fmt="ro")
    axs9.errorbar(urunNumberArr,Pt_Arr,yerr=Pt_err_Arr,label="No FCup weights",fmt="ko")
    axs9.plot(_rcdb_run[boolTpolPos],_rcdb_Tpol[boolTpolPos],"k*",label=r"RCDB Tpol$>$0",markersize=10,fillstyle='none')
    axs9.plot(_rcdb_run[boolTpolNeg],np.abs(_rcdb_Tpol[boolTpolNeg]),"r*",label=r"RCDB Tpol$<$0",markersize=10,fillstyle='none')
    axs9.grid()
    #xHarut=[17237,17262,17265,17269]
    #yHarut=[0.231949,0.18501,0.19945,0.15181]
    #yerrHarut=[0.02,0.02,0.02,0.02]
    #axs9.errorbar(xHarut,yHarut,yerr=yerrHarut,fmt="bs",label="Harut's results")
    axs9.legend()
    axs9.set_xlabel("Run",fontsize=20)
    axs9.set_ylabel(r"$P_{t}$",fontsize=30)
    axs9.text(0.15,0.25,r"$P_b={:.1f}\%$".format(100*beamPol),fontsize=25,horizontalalignment='center',
     verticalalignment='center',transform=axs9.transAxes)
    axs9.set_ylim(0,1)
    
    
    # N+ - N-
    runs = []
    vals = []
    errvals = []
    for r in urunNumberArr:
        runs.append(r)
        dff = df[df["Run"]==r]
        vals.append(np.sum((dff["N+"]-dff["N-"]))/np.sum((dff["N+"]+dff["N-"])))
        errvals.append(2/(np.sum(dff["N+"])+np.sum(dff["N-"]))**2*np.sqrt(np.sum(dff["N+"])**2*np.sum(dff["N-"])+np.sum(dff["N-"])**2*np.sum(dff["N+"])))
    fig10,axs10 = plt.subplots(1,1,figsize=(12,10))
    axs10.ticklabel_format(useOffset=False)
    axs10.errorbar(runs,vals,yerr=errvals,fmt="ro")
    axs10.set_xlabel("Run",fontsize=20)
    axs10.set_ylabel(r"$\frac{N^{+}-N^{-}}{N^{+}+N^{-}}$",fontsize=30)
    axs10.grid()
        
    # Return stuff
    DF = df.copy()
    DF["Pb"]=np.ones(reps*nruns)*beamPol
    DF["Pt"]=np.repeat(Pt_w_Arr,reps)
    DF["Pt_err"]=np.repeat(Pt_w_err_Arr,reps)
    DF.rename(columns={'Tpol':'Tpol RCDB'}, inplace=True)
    return DF

In [21]:
def get_table(directory, min_run, max_run, target):
    
    # Get root files for analysis
    rootfiles=get_files_in_range(directory, min_run, max_run)
    
    # Get individual polarizations
    ret = []
    for f in rootfiles:
        vals=get_single_run_hists(f, target)
        print("Completed run:",vals[0])
        ret.append(vals)
    
    # Get unpacked dataframe
    df = unpack_to_dataframe(ret)
    
    return df

In [ ]:
get_table("/volatile/clas12/users/gmat/clas12analysis.sidis.data/rgc/tpol/data/8.3.4/",16500,16600,"ND3")